In [1]:
import k3d
import vtk
import glob
import os
from vtk.util import numpy_support
import numpy as np
from tqdm import tqdm

visc_dyn = 0.003

In [2]:
reader = vtk.vtkSTLReader()
reader.SetFileName('./tetniak/c0006.stl')
reader.Update()
geometry_wss = reader.GetOutput()

In [3]:
# normalGenerator = vtk.vtkPolyDataNormals()
# normalGenerator.SetInputData(geometry_wss)
# normalGenerator.ComputePointNormalsOn()
# normalGenerator.ComputeCellNormalsOff()
# normalGenerator.SetSplitting(0)
# normalGenerator.SetConsistency(0)
# normalGenerator.Update()

# geometry_wss = normalGenerator.GetOutput()
# normals = numpy_support.vtk_to_numpy(geometry_wss.GetPointData().GetNormals())
# vertices = numpy_support.vtk_to_numpy(geometry_wss.GetPoints().GetData())

# wss = {}
        
# for iteration, vti_file in tqdm(enumerate(glob.glob('./tetniak/*.vti'))):
#     reader = vtk.vtkXMLImageDataReader()
#     reader.SetFileName(vti_file)
#     reader.Update()
#     vti = reader.GetOutput()
#     du = vti.GetSpacing()[0] * np.sqrt(3) * 2
    
#     points = vtk.vtkPoints()
    
#     for normal, pos in zip(normals, vertices):
#         points.InsertNextPoint(pos + normal * (-du))
    
#     pointsPolyData = vtk.vtkPolyData()
#     pointsPolyData.SetPoints(points)
#     probe_filter = vtk.vtkProbeFilter()
#     probe_filter.SetInputData(pointsPolyData)
#     probe_filter.SetSourceData(vti)
#     probe_filter.GetOutputPort()
#     probe_filter.Update()

#     probed_data = probe_filter.GetOutput().GetPointData()

#     lbm_v = numpy_support.vtk_to_numpy(probed_data.GetArray('v [m/s]'))
    
#     lbm_v = lbm_v * visc_dyn / du
#     lbm_v[lbm_v > 1e35] = np.nan
#     lbm_v[lbm_v < -1e35] = np.nan
    
#     wss[iteration] = np.linalg.norm(lbm_v.reshape(-1,3), axis=1)    

# np.save('./tetniak/wss.npy', wss) 

In [5]:
wss = np.load('./tetniak/wss.npy',allow_pickle=True).item()

In [6]:
wss_reduced = {}

for k in wss.keys():
    if k % 4 == 0:
        wss_reduced[str(k * 0.01)] = wss[k]
        
print(len(wss_reduced.keys()))

280


In [12]:
wss_reduced


{'0.0': array([1.5421548e-06, 1.5123132e-06, 1.6132074e-06, ..., 1.0752692e-06,
        1.1669053e-06, 1.1906480e-06], dtype=float32),
 '0.04': array([6.0295320e-06, 1.4690758e-05, 1.3500272e-06, ..., 1.1075865e-05,
        9.8292376e-06, 8.5818183e-06], dtype=float32),
 '0.08': array([8.57934992e-06, 1.20963959e-05, 7.42793191e-06, ...,
        1.07434125e-05, 7.93960317e-06, 7.96340737e-06], dtype=float32),
 '0.12': array([6.4828919e-06, 4.6872788e-06, 1.2052532e-05, ..., 8.4699996e-06,
        1.0076800e-05, 9.3161379e-06], dtype=float32),
 '0.16': array([4.47178945e-06, 8.39199129e-06, 4.18088484e-06, ...,
        1.46461525e-05, 1.39423446e-05, 1.01784599e-05], dtype=float32),
 '0.2': array([3.0110671e-06, 8.1018052e-06, 3.5908511e-06, ..., 8.1794269e-06,
        1.5996542e-05, 1.3290736e-05], dtype=float32),
 '0.24': array([8.5034571e-06, 1.1139898e-05, 5.6809522e-06, ..., 1.5054290e-05,
        2.0215797e-05, 1.1167266e-05], dtype=float32),
 '0.28': array([7.3925598e-06, 9.35261

In [14]:
vertices = numpy_support.vtk_to_numpy(geometry_wss.GetPoints().GetData())
indices = numpy_support.vtk_to_numpy(geometry_wss.GetPolys().GetData()).astype(np.uint32).reshape(-1, 4)[:, 1:4]

plot = k3d.plot()
plot += k3d.mesh(vertices, indices, 
                 color_map=k3d.colormaps.paraview_color_maps.Jet, 
                 attribute=wss_reduced, 
                 color_range=[0,3], 
                 )
plot.display()

Output()

In [10]:
plot.camera_auto_fit = False
plot.grid_auto_fit = False

In [11]:
from time import sleep

for t in range(1120):
    plot.time = t / 100.0
    sleep(1.0/30.0)

11.19